In [1]:
from builtins import Exception
import urllib3
from bs4 import BeautifulSoup
from requests import get
import mysql.connector
import re
import datetime

# Class Liputan6 Scraping

class dibawah merupakan proses pengambilan data dari website berita Liputan6.com yang dimana pemanggilan semua fungsi dibawah tersebut nantinya akan dipanggil pada fungsi [scrapingDataLiputan6]

In [8]:
class Liputan6:
     #deklarasi variable global
    url = ""
    mydb = ""
    keyword = ""
    urlDetailData = ""
    date = ""
    
    #fungsi dari inisialisasi data saat memanggil class pertama kali
    def __init__(self, urlBerita, pencarian,searchDate):
        self.url = urlBerita
        self.keyword = pencarian
        try:
            if(searchDate == ""):
                self.date = datetime.datetime.now()
            else:
                self.date = datetime.datetime.strptime(searchDate, '%d-%m-%Y')
        except Exception as e:
            print(e)
        self.mydb = mysql.connector.connect(host="localhost",user="root", password="",database="db-berita-mca")
    
    #fungsi menyimpan data ke database
    def simpanDatabase(self,data):
        mycursor = self.mydb.cursor()
        sql = "INSERT INTO tb_berita (media,judul_berita,label,penulis,release_date,url,content) VALUES (%s,%s,%s,%s,%s,%s,%s)"
        mycursor.execute(sql, data)
        self.mydb.commit()
        return
    
    #fungsi buat validasi data duplicate di database
    def validasiDataDatabase(self,dataBerita):
        mycursor = self.mydb.cursor()
        sql = "SELECT * FROM tb_berita WHERE url = %s"
        dataUrl = dataBerita
        url = (dataBerita,)
        mycursor.execute(sql, url)
        myresult = mycursor.fetchall()
        return myresult
        
    def getUrlDetail(self,page):
        self.urlDetailData = []
        for page in range(page): 
            try:
                if(self.keyword == ""):
                    urlGet = (self.url+"/news/indeks/"+self.date.strftime("%Y/%m/%d")+"?page={}".format(page+1))
                    soup = BeautifulSoup(get(urlGet).text, 'lxml')
                    getContent = soup.find('article','main')
                    groupContent = getContent.find_all('h4','articles--rows--item__title')
                    for x in groupContent:
                        news_url = x.find('a', href=True).get('href')
                        mergeUrl = (news_url+"?page=all")
                        self.urlDetailData.append(mergeUrl)
                else:
                    dateNow = self.date.strftime("%Y/%m/%d")
                    date = dateNow.replace('/','%2F')
                    urlGet = (self.url+"/search?order=latest&chammel_id=&from_date="+date+"&to_date="+date+"&type=all&q="+self.keyword)
                    print(urlGet)
                    soup = BeautifulSoup(get(urlGet).text, 'lxml')
                    getContent = soup.find('article','main')
                    groupContent = getContent.find_all('h4','articles--iridescent-list--text-item__title')
                    for x in groupContent:
                        news_url = x.find('a', href=True).get('href')
                        mergeUrl = (news_url+"?page=all")
                        self.urlDetailData.append(mergeUrl)
            except Exception as e:
                    continue 
                    
                    
    def scrapingLiputan6(self):
        self.getUrlDetail(3)
        for dataBerita in self.urlDetailData:
            myresult = self.validasiDataDatabase(dataBerita)
            if(len(myresult)==0):
                try:
                    media = "Liputan6.com"
                    web_data = get(dataBerita)
                    soup = BeautifulSoup(web_data.text, 'html.parser')
                    page = soup.find('div','inner-container-article')
                    title = page.find('h1','read-page--header--title entry-title').text
                    labelGroup = page.find_all('a','read-page--breadcrumb--item__title')
                    for x in labelGroup:
                        label = x.find('span').text
                    author = page.find('span','read-page--header--author__name fn').text
                    release_date = page.find('time','read-page--header--author__datetime updated').text      
                    groupContent = page.find('div','read-page--content')
                    texts = groupContent.find_all('p')
                    news_content = ' '.join([text.text for text in texts])           
                    data = (media,title,label,author,release_date,dataBerita,news_content)
                    self.simpanDatabase(data)
                except Exception as e:
                    continue 
            else:
                continue

In [9]:
url = 'https://www.liputan6.com'
pencarian = 'vaksin'
tanggal = '12-12-2021'
testing = Liputan6(url, pencarian,tanggal)
testing.getUrlDetail(4)
testing.urlDetailData

https://www.liputan6.com/search?order=latest&chammel_id=&from_date=2021%2F12%2F12&to_date=2021%2F12%2F12&type=all&q=vaksin
https://www.liputan6.com/search?order=latest&chammel_id=&from_date=2021%2F12%2F12&to_date=2021%2F12%2F12&type=all&q=vaksin
https://www.liputan6.com/search?order=latest&chammel_id=&from_date=2021%2F12%2F12&to_date=2021%2F12%2F12&type=all&q=vaksin
https://www.liputan6.com/search?order=latest&chammel_id=&from_date=2021%2F12%2F12&to_date=2021%2F12%2F12&type=all&q=vaksin


['https://www.liputan6.com/news/read/4833626/infografis-seruan-who-akhiri-pandemi-covid-19-di-2022?page=all',
 'https://www.liputan6.com/bola/read/4833494/bola-ganjil-tidak-ada-yang-lebih-menderita-dari-timnas-indonesia-di-piala-aff?page=all',
 'https://www.liputan6.com/news/read/4767705/infografis-sudah-vaksinasi-covid-19-yuk-tetap-taat-protokol-kesehatan?page=all',
 'https://www.liputan6.com/global/read/4834800/wabah-flu-burung-dahsyat-di-israel-bunuh-2000-bangau-liar?page=all',
 'https://www.liputan6.com/tekno/read/4834822/subscriber-youtube-fuji-an-naik-drastis-dari-473-ribu-ke-117-juta-dalam-2-hari?page=all',
 'https://www.liputan6.com/saham/read/4834824/ihsg-lanjutkan-penguatan-usai-libur-natal-2021-investor-buru-saham-bmri-hingga-tkim?page=all',
 'https://www.liputan6.com/tekno/read/4834821/sms-pertama-di-dunia-dijual-dalam-bentuk-nft-senilai-rp-17-miliar?page=all',
 'https://www.liputan6.com/regional/read/4834820/omicron-di-indonesia-bikin-khawatir-menko-luhut-imbau-warga-libur